In [128]:
import numpy as np
import pandas as pd
import MeCab
import json
import re

In [10]:
flatten = lambda l: [i for sub_l in l for i in sub_l]

m_parser = MeCab.Tagger("-u /Users/dontsentouin/mecab-user-dic/compound.dic -Owakati")
def parse(s):
    words = []
    node = m_parser.parseToNode(s)
    while node:
        if len(node.surface) is 0:
            node = node.next
            continue

        words.append(node.surface)
        node = node.next
    
    return words

In [7]:
with open("../../data/compound_train.json", 'r') as f:
    raw_train = json.load(f)

In [161]:
train_df = pd.read_pickle("../../data/Production_train_split_word_with_repl_compound.pkl")
train_df._id = train_df._id.astype(str)
train_df.head()

,_id,label,sentence,title,words,repl_words
0,10166,False,アンモニア (英: ammonia) は分子式が NH 3 で表される無機化合物。,アンモニア,"[アンモニア, (, 英, :, ammonia, ), は, 分子, 式, が, NH, ...","[[title-compound], (, 英, :, [title-compound], ..."
1,10166,False,常温常圧では無色の気体で、特有の強い刺激臭を持つ。,アンモニア,"[常温, 常, 圧, で, は, 無色, の, 気体, で, 、, 特有, の, 強い, 刺...","[常温, 常, 圧, で, は, 無色, の, 気体, で, 、, 特有, の, 強い, 刺..."
2,10166,False,水に良く溶けるため、水溶液（アンモニア水）として使用されることも多く、化学工業では基礎的な窒...,アンモニア,"[水, に, 良く, 溶ける, ため, 、, 水溶液, （, アンモニア水, ）, として,...","[水, に, 良く, 溶ける, ため, 、, 水溶液, （, [compound], ）, ..."
3,10166,False,塩基の程度は水酸化ナトリウムより弱い。,アンモニア,"[塩基, の, 程度, は, 水酸化ナトリウム, より, 弱い, 。]","[塩基, の, 程度, は, [compound], より, 弱い, 。]"
4,10166,False,窒素原子上の孤立電子対のはたらきにより、金属錯体の配位子となり、その場合はアンミンと呼ばれる。,アンモニア,"[窒素, 原子, 上, の, 孤立, 電子, 対, の, はたらき, により, 、, 金属,...","[窒素, 原子, 上, の, 孤立, 電子, 対, の, はたらき, により, 、, 金属,..."


In [160]:
def tagging(s, train):
    train.sort(key=len, reverse=True)
    tag_seq = ['O'] * len(s)
    
    if len(train) is 0:
        return tag_seq
    
    for train_s in train:
        if len(train_s) > len(s):
            continue
            
        for i in range(0, len(s) - len(train_s)):
            start = i
            end = i + len(train_s)
            if s[start:end] == train_s:
                tag_seq = _tagging(tag_seq, start, end)
                
    return tag_seq

def _tagging(tag_seq, start, end):
    _tag_seq = tag_seq.copy()
    _tag_seq[start] = 'B'
    
    if (end - start) > 1:
        if _tag_seq[end - 1] == 'O':
            _tag_seq[end - 1] = 'E'
            
    if (end - start) > 2:
        if not ('B' in _tag_seq[start + 1:end - 1] or 'E' in _tag_seq[start + 1:end - 1]):
            _tag_seq[start + 1:end - 1] = ['I'] * len(_tag_seq[start + 1:end - 1])
    
    return _tag_seq

In [162]:
# タグ付け

train_df = train_df.assign(tag=np.nan)

for entry in raw_train['entry']:
    _id = str(entry['WikipediaID'])
    productions = entry['Attributes']['製造方法']
    
    if len(productions) is 0:
        train_df.loc[train_df._id == _id, 'tag'] = \
            train_df.loc[train_df._id == _id].words.apply(lambda x: ['O'] * len(x))
        continue
        
    train_sents = flatten([re.findall(r'[^。]+(?:。|$)', production) for production in productions])
    train_split_sents = [parse(s) for s in train_sents]
    
    train_df.loc[train_df._id == _id, 'tag'] = \
            train_df.loc[train_df._id == _id].words.apply(lambda x: tagging(x, train_split_sents))

In [163]:
train_idx = pd.read_csv("../../data/train.csv")._id.astype(str).unique()
test_idx = pd.read_csv("../../data/valid.csv")._id.astype(str).unique()

In [164]:
train_df.loc[train_df._id.isin(train_idx)].to_pickle("../../data/Production_train_repl_compound.pkl")
train_df.loc[train_df._id.isin(test_idx)].to_pickle("../../data/Production_test_repl_compound.pkl")